#### 전선 데이터셋

#### 라이브러리 불러오기

In [3]:
import copy
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from aiddd.data_manager import read_data, write_data
from freeman.plt_setting import plt_settings

plt_settings()

In [7]:
_start = datetime.now()
df_data = read_data('2nd LINE')
print(
    f'Total Elapsed Time for Data Load: {datetime.now()-_start}, '
    f'Shape: {df_data.shape}'
)

Total Elapsed Time for Data Load: 0:00:22.138188, Shape: (39861, 77)


#### 데이터 확인

In [8]:
# 유일한 값으로 구성된 컬럼은 제거(학습에 필요없음)
unique_cnt_cols = df_data.nunique()
single_value_cols = unique_cnt_cols[unique_cnt_cols <= 1].index
df_data.drop(columns=single_value_cols, axis=1, inplace=True)
print(
    f'Number of Drop Columns: {len(single_value_cols)}\n'
    f'Columns: {[col for col in single_value_cols]}'
)

Number of Drop Columns: 40
Columns: ['공사변경순번', '공사구분코드', '전압구분코드', '공사성격코드', '회선명', '상배열구분코드', '전선조정코드1', '전선공사특성코드2', '전선조정코드2', '전선펴기여부', '중성선조정코드', '직선접속공사특성코드', '슬리브접속수량', '시공방법코드', '건물방호관코드', '건물방호관수량', '고객제공여부', '직선접속중성선공사성격코드', '전압선중성선사용여부', '중성선전압선사용여부', '전선펴기긴선여부', '기설전선여부', '전선공사특성코드3', '전선종류코드3', '전선규격코드3', '전선구분코드3', '전선조정코드3', '기설설비유실여부', '사업소코드', '변전소코드', '회선코드', '기설설비상태코드', '종단접속공사코드', '종단접속수량', '조가형태공사코드', '조가형태코드', '파형관규격코드', '디지털장력계여부', '케이블헤더수량', 'NDIS사업소코드']


In [9]:
# 공사번호별 레코드수 확인
df_data['공사번호'].value_counts()

# 전선수가 1개 이상 11개 미만인 데이터만 대상으로 작업할 예정

공사번호
476920194408    49
477420193177    49
474920215348    46
476920213694    38
475920213201    36
                ..
442720233173     1
474920214500     1
474920214502     1
442720233163     1
474620234659     1
Name: count, Length: 17191, dtype: int64

In [10]:
# 접수종류명 확인
df_data['접수종류명'].value_counts()

# '신설(상용/임시)'만 대상으로 작업할 예정

접수종류명
신설(상용/임시)     33945
신설(발전기)        4218
공급방식변경증설        624
증설              308
단순공급방식변경        273
계약종별변경증설        184
해지후재사용(신규)      115
공급지점변경          111
계약단위합병           40
신증설(예비전력)        22
일부해지(신규)         14
계약단위분할            7
Name: count, dtype: int64

In [11]:
# 계약전력 확인
df_data['계약전력'].value_counts()

계약전력
3      15965
5      11190
4       1782
10      1158
100     1047
       ...  
385        1
173        1
138        1
124        1
156        1
Name: count, Length: 200, dtype: int64

In [12]:
# 좌표확인
df_data['X좌표-Y좌표'].head(10)

0    14289819.0189981,4448996.33479691,14289818.237...
1    14289811.3602924,4449112.09869807,14289810.953...
2    14289796.7644173,4449178.15244157,14289796.308...
3    14289874.1009936,4448903.55163219,14289872.802...
4    14289831.244884,4448951.31661164,14289830.7514...
5    14289801.7912967,4449034.10434287,14289802.020...
6    14289868.401279,4447980.66854729,14289867.8549...
7    14289855.0915732,4448036.95675113,14289854.803...
8    14289844.103554,4448109.84989056,14289843.9521...
9    14289842.5975731,4448148.69257442,14289844.016...
Name: X좌표-Y좌표, dtype: object

In [15]:
# 좌표 갯 수 확인
df_data['좌표리스트'] = df_data['X좌표-Y좌표'].str.split(',')
df_data['좌표수'] = df_data['좌표리스트'].apply(len)

In [14]:
location_lens = df_data['X좌표-Y좌표'].str.len()

In [22]:
import pyproj

# 입력 좌표와 대상 좌표계 정의
input_coordinates = (14289819.0189981, 4448996.33479691)
input_epsg = pyproj.CRS.from_epsg(5179)  # 입력 좌표의 좌표계 (UTM Zone 52N)
output_epsg = pyproj.CRS.from_epsg(4326)  # 대상 좌표계 (WGS 84, EPSG 4326)

# 좌표 변환
transformer = pyproj.Transformer.from_crs(input_epsg, output_epsg, always_xy=True)
output_coordinates = transformer.transform(*input_coordinates)

# 경도와 위도 추출
longitude, latitude = output_coordinates

# 결과 출력
print(f"경도 (Longitude): {longitude:.6f}")
print(f"위도 (Latitude): {latitude:.6f}")

경도 (Longitude): -149.878624
위도 (Latitude): 11.939015
